<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/JJ/JL_T5_huggingface_abstraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup

In [1]:
! pip install transformers datasets #huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 441 kB 34.4 MB/s 
     |████████████████████████████████| 7.6 MB 43.2 MB/s 
     |████████████████████████████████| 163 kB 48.3 MB/s 
     |████████████████████████████████| 212 kB 42.8 MB/s 
     |████████████████████████████████| 115 kB 38.4 MB/s 
     |████████████████████████████████| 127 kB 15.7 MB/s 
     |████████████████████████████████| 115 kB 46.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [24]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [3]:
import transformers

print(transformers.__version__)

# Verify! Version must be at least 4.16.0

4.23.1


In [26]:
from datasets import load_dataset, load_metric, load_dataset, load_from_disk, load_dataset_builder

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf

from pprint import pprint

In [9]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Notebook-Level Settings

In [33]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "google/t5-v1_1-base"
tokenizer_checkpoint = "t5-base" # because it threw an error with the model checkpoint, but I believe it's the same tokenizer
batch_size = 8

## Helper Functions

In [19]:
def summarize_dataset (dataset, config=None):
    builder = load_dataset_builder(dataset, config)
    pprint(f"Description:\n {builder.info.description}")
    print(f"Features:")
    pprint(builder.info.features)
    return

In [20]:
def word_count(string):
  return(len(string.strip().split(" ")))

# Loading SQuAD

## From `datasets` or Google Drive

In [12]:
# Uncomment accordingly
# datasets = load_dataset('squad_v2' if squad_v2 else 'squad')
datasets = load_from_disk("/content/drive/MyDrive/w266 NLP Final Project/Data/squad.hf")

In [13]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [14]:
# Look at first example
pprint(datasets['train'][0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

In [16]:
# Shuffle dataset and select specific number of examples for development

data_count = 250
sample = datasets['train'].shuffle(seed=1962).select(range(data_count))
df = pd.DataFrame()
df['answer'] = [answer['text'][0] for answer in sample['answers']]
df['context'] = sample['context']
df['question'] = sample['question']

In [17]:
df

,answer,context,question
0,biotech companies,"Prior to moving its headquarters to Chicago, a...",What type of businesses did Nickles want to at...
1,Tytus Woyciechowski,Four boarders at his parents' apartments becam...,To whom did Chopin reveal in letters which par...
2,the Endangered Species Committee,The question to be answered is whether a liste...,"If a species may be harmed, who holds final sa..."
3,China,"In Asian countries such as China, Korea, and J...",What country has the dog as part of its 12 ani...
4,45 years,Saint Athanasius of Alexandria (/ˌæθəˈneɪʃəs/;...,How long did his episcopate last?
...,...,...,...
245,"December 8, 1991","On June 12, 1990, the Congress of People's Dep...",On what date were the Belavezha Accords signed?
246,the Boreal Kingdom,"Phytogeographically, Greece belongs to the Bor...",Greece's plant distribution belongs to what?
247,May to September,"Fog is fairly common, particularly in spring a...",What months do thunderstorms occur in Boston?
248,the corporation,The creation of a modern industrial economy to...,What became the dominant form of business orga...


In [21]:
max([word_count(x) for x in df.context])

297

# Preprocess training data
## Tokenization

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [31]:
# Check if it's a "fast" tokenizer

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [32]:
# Test tokenizer
tokenizer('hello, world!')

{'input_ids': [21820, 6, 296, 55, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
# Tokenization settings
max_length = 